In [7]:
# PART ONE: Data Reading
import pandas as pd
import numpy as np
from scipy.stats import skew
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
def read_data():
    #step1:reading csv data
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    #train.head()   # take a brief look at training data
    all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition'])) # concat training&test data
    return train,test,all_data
#train,test,all_data = read_data()
def model_input():
    alldata_after_filling_missing_skew = pd.read_csv('../input/alldata_after_filling_missing_skew.csv')
    y = pd.read_csv('../input/train_label_skew',header=None)
    test_label = pd.read_csv('../input/test_id',header=None)
    return alldata_after_filling_missing_skew,y,test_label
#alldata_after_filling_missing_skew,y,test_label = model_input()

In [2]:
'''
the preprocessing apart from dimensional reduction is :
@ missing data filling: 
    RandomForest filling for important features(highly relevant to SalePrice);
    another value filling for features having many missing value, or its missing value has some meaning.
    mean()/mode() filling for other features
@ data transform: get_dummies for categorical features
@ log transform for SalePrice and some skewed features
''' 
#alldata_after_filling_missing_skew = pd.read_csv('../input/alldata_after_filling_missing_skew.csv')
#y = pd.read_csv('../input/train_label_skew',header=None)
#import xgboost as xgb
#model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
#model_xgb.fit(alldata_after_filling_missing_skew.iloc[:1460],y)
#pre_val = np.expm1(pd.DataFrame(model_xgb.predict(alldata_after_filling_missing_skew.iloc[1460:])))
#test_label = pd.read_csv('../input/test_id',header=None)
#result = pd.DataFrame()
#result['Id'] = test_label[0]
#result['SalePrice'] = pre_val[0]
#result.to_csv('../input/result_xgb_1123_nofeaselec_skew.csv',index=None)

'\nthe preprocessing apart from dimensional reduction is :\n@ missing data filling: \n    RandomForest filling for important features(highly relevant to SalePrice);\n    another value filling for features having many missing value, or its missing value has some meaning.\n    mean()/mode() filling for other features\n@ data transform: get_dummies for categorical features\n@ log transform for SalePrice and some skewed features\n'

In [4]:
# PART FOUR: Feature Decomposition

from sklearn.decomposition import PCA
# Note that the input df must not have categorial features or missing value,
# do this after preprocessing fo filling missing value and feature transformation
def pca_reduc(df, num_fea_toleave='mle'):
    # @return type: pd.DataFrame
    pca = PCA(n_components=num_fea_toleave)
    after_pca = pca.fit_transform(df)
    print 'Percentage of variance explained by each of the selected components:'
    print(pca.explained_variance_ratio_)
    #print pd.DataFrame(new).info()
    return pd.DataFrame(after_pca)
#all_data = dummy_all(all_data)
#all_data.fillna(all_data.mean(),inplace=True)
#all_data = pca_reduc(all_data,30)
#all_data.info(verbose=True, max_cols=1000)

from sklearn.decomposition import KernelPCA
# Kernel PCA ==> non-linear dimensionality reduction through the use of kernels
# Somewhat like kernel in SVM
# kernel = “linear” | “poly” | “rbf” | “sigmoid” | “cosine” | “precomputed”
def kernelpca_reduc(df, kernel='linear',num_fea_toleave=50):
    kpca = KernelPCA(n_components=num_fea_toleave,kernel = kernel,n_jobs=-1)
    after_kpca = kpca.fit_transform(df)
    print 'the selected features Eigenvalues in decreasing order:'
    print (kpca.lambdas_)
    return pd.DataFrame(after_kpca)
#all_data = dummy_all(all_data)
#all_data.fillna(all_data.mean(),inplace=True)
#all_data = kernelpca_reduc(all_data,kernel='rbf',num_fea_toleave=50)
#print all_data.shape

from sklearn.decomposition import TruncatedSVD
# Dimensionality reduction using truncated SVD
def truncatedSVD_reduc(df,num_fea_toleave=50):
    # provide a random_state to get stable output
    svd = TruncatedSVD(n_components=num_fea_toleave, n_iter=7, random_state=42)
    after_trans = svd.fit_transform(df)
    print 'Percentage of variance explained by each of the selected components:'
    print(svd.explained_variance_ratio_) 
    return pd.DataFrame(after_trans)
#all_data = dummy_all(all_data)
#all_data.fillna(all_data.mean(),inplace=True)
#all_data = truncatedSVD_reduc(all_data,num_fea_toleave=50)
#print all_data.shape

In [6]:
# PART FIVE: Feature Selection

from sklearn.feature_selection import RFECV
# RFECV: Feature ranking with recursive feature elimination and cross-validated selection of the best number of features.
def fea_sel_rfecv(train_x,train_y,test_x,estimator):
    rfecv = RFECV(estimator=estimator,scoring='neg_mean_squared_error',n_jobs=-1)
    after_d = rfecv.fit_transform(train_x,train_y)
    print("Optimal number of features : %d" % rfecv.n_features_)
    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score(neg_mean_squared_error)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    return pd.DataFrame(after_d),pd.DataFrame(rfecv.transform(test_x))
#alldata_nomissing = pd.read_csv('../input/alldata_after_filling_missing.csv')
#from sklearn import svm
#clf = svm.LinearSVR()
#after_d,after_d_test = (fea_sel_rfecv(alldata_nomissing.iloc[:1460],train['SalePrice'],alldata_nomissing.iloc[1460:],clf))
#print after_d.shape
#print after_d_test.shape

from sklearn.feature_selection import SelectFromModel
# u can see from 'SelectFromModel' that this method use model result to select features, 'Wrapper'
# estimator: a supervised model with fit() method
def fea_sel_tree(train_x,train_y,estimator):
    estimator = estimator.fit(train_x,train_x)
    print 'feature importances in this model',
    print sorted(estimator.feature_importances_,reverse=True)
    model = SelectFromModel(estimator,prefit = True)
    after_sel = model.transform(train_x)
    return pd.DataFrame(after_sel)
#train = dummy_all(train)
#train.fillna(train.mean(),inplace=True)
#from sklearn.ensemble import RandomForestRegressor
#clf = RandomForestRegressor(random_state=0,n_estimators=50)
#print fea_sel_tree(train.iloc[:,1:-1],train['SalePrice'],clf).shape

In [18]:
from sklearn.model_selection import cross_val_score
alldata_after_filling_missing_skew,y,test_label = model_input()
def rmse_cv(model, data_x, data_y):
    rmse= np.sqrt(-cross_val_score(model, data_x, data_y, scoring="mean_squared_error", cv = 3))
    return(rmse)

In [19]:
alldata_after_filling_missing_skew,y,test_label = model_input()
num_fea_toleave = 200  # need 
alldata_rec = pca_reduc(alldata_after_filling_missing_skew,num_fea_toleave)
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
print rmse_cv(model_xgb,alldata_rec.iloc[:1460],y)

Percentage of variance explained by each of the selected components:
[  9.66823521e-01   2.47076500e-02   4.61466268e-03   2.38038610e-03
   2.57704922e-04   1.78065391e-04   1.53568252e-04   1.21621284e-04
   1.17793406e-04   8.43552745e-05   7.28115983e-05   5.96685672e-05
   5.26004969e-05   4.11435019e-05   3.52181392e-05   3.19541174e-05
   1.99598446e-05   1.88707986e-05   1.74078551e-05   1.17572974e-05
   1.06521573e-05   1.03831585e-05   8.72509347e-06   8.38515095e-06
   7.99378491e-06   7.08155812e-06   6.23161041e-06   5.65530391e-06
   5.29206753e-06   4.95437884e-06   4.75773594e-06   4.54058438e-06
   4.34970503e-06   3.78027993e-06   3.68309566e-06   3.56545801e-06
   3.42067221e-06   3.29100327e-06   3.13612475e-06   3.05174734e-06
   2.84474246e-06   2.74807881e-06   2.65150242e-06   2.59402129e-06
   2.51982854e-06   2.45756827e-06   2.39573910e-06   2.29183833e-06
   2.15166315e-06   2.12193072e-06   2.05204519e-06   1.96357995e-06
   1.83561234e-06   1.76543061e-06

In [13]:
alldata_after_filling_missing_skew,y,test_label = model_input()
num_fea_toleave = 10  # need 
alldata_rec = kernelpca_reduc(all_data,kernel='rbf',num_fea_toleave=50)
import xgboost as xgb
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
print rmse_cv(model_xgb,alldata_rec.iloc[:1460],y)

Percentage of variance explained by each of the selected components:
[  9.66823521e-01   2.47076500e-02   4.61466268e-03   2.38038610e-03
   2.57704922e-04   1.78065391e-04   1.53568252e-04   1.21621284e-04
   1.17793406e-04   8.43552745e-05]
[ 0.11345483  0.1393913   0.12302224]


In [ ]:
# actually, we can do some operations on features, and get more features to select.
# Its shown that results from features may also work.
# for example, alpha = num_buy/num_click do means something in shopping website's analysis.